In [1]:
import pandas as pd
from pycaret.regression import *

In [2]:
train = pd.read_csv("train.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)

In [3]:
# おそらく不要
train['country'] = train['country'].apply(str)
train['store'] = train['store'].apply(str)
train['product'] = train['product'].apply(str)


In [6]:
train

,date,country,store,product,num_sold
row_id,,,,,
0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911
...,...,...,...,...,...
26293,2018-12-31,Sweden,KaggleMart,Kaggle Hat,823
26294,2018-12-31,Sweden,KaggleMart,Kaggle Sticker,250
26295,2018-12-31,Sweden,KaggleRama,Kaggle Mug,1004


In [8]:
exp1 = setup(train, target = 'num_sold',
             categorical_features=["country", "store","product"])

,Description,Value
0,session_id,7949
1,Target,num_sold
2,Original Data,"(26298, 5)"
3,Missing Values,False
4,Numeric Features,0
5,Categorical Features,3
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(18408, 28)"


In [9]:
# モデルの比較
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,40.1492,6185.3302,78.4537,0.9132,0.1335,0.0970,0.1150
rf,Random Forest Regressor,42.8491,7002.8735,83.5164,0.9017,0.1436,0.1033,1.4290
dt,Decision Tree Regressor,43.0696,7080.5411,83.9876,0.9006,0.1452,0.1042,0.0440
et,Extra Trees Regressor,43.0821,7097.7182,84.0826,0.9004,0.1449,0.1041,1.8020
gbr,Gradient Boosting Regressor,44.8800,7177.9563,84.5464,0.8993,0.1511,0.1134,0.5970
knn,K Neighbors Regressor,45.0836,7849.1014,88.4388,0.8898,0.1530,0.1090,0.4400
ridge,Ridge Regression,83.0424,14987.6475,122.3266,0.7895,0.5371,0.2917,0.0230
lar,Least Angle Regression,83.0565,14987.6703,122.3267,0.7895,0.5371,0.2919,0.0260
br,Bayesian Ridge,83.0330,14987.6410,122.3265,0.7895,0.5371,0.2917,0.0490
lasso,Lasso Regression,81.6176,15218.0301,123.2555,0.7863,0.5531,0.2751,0.0270


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=7949, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [10]:
# Light Gradient Boosting Machineのモデルを作成
rf = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,38.9397,5870.7651,76.6209,0.9135,0.1317,0.0951
1,40.2971,4912.0191,70.0858,0.9275,0.1334,0.0990
2,39.0716,4795.3220,69.2483,0.9287,0.1356,0.0993
3,41.3977,6253.7034,79.0804,0.9206,0.1326,0.0976
4,38.2620,6169.1588,78.5440,0.9093,0.1309,0.0941
5,40.9933,5755.9820,75.8682,0.9193,0.1386,0.0981
6,40.5548,7376.7174,85.8878,0.8969,0.1330,0.0960
7,41.2921,7031.5243,83.8542,0.9009,0.1342,0.0991
8,40.5813,6345.6419,79.6595,0.9156,0.1320,0.0953
9,40.1027,7342.4685,85.6882,0.8994,0.1335,0.0968


In [13]:
# 解析結果の確認
evaluate_model(rf)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [14]:
#モデルの確定
final_rf = finalize_model(rf)
print(final_rf)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=7949, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [35]:
#予測の実行
unseen_predictions = predict_model(final_rf, data = test)
print(test)

              date  country       store         product
row_id                                                 
26298   2019-01-01  Finland  KaggleMart      Kaggle Mug
26299   2019-01-01  Finland  KaggleMart      Kaggle Hat
26300   2019-01-01  Finland  KaggleMart  Kaggle Sticker
26301   2019-01-01  Finland  KaggleRama      Kaggle Mug
26302   2019-01-01  Finland  KaggleRama      Kaggle Hat
...            ...      ...         ...             ...
32863   2019-12-31   Sweden  KaggleMart      Kaggle Hat
32864   2019-12-31   Sweden  KaggleMart  Kaggle Sticker
32865   2019-12-31   Sweden  KaggleRama      Kaggle Mug
32866   2019-12-31   Sweden  KaggleRama      Kaggle Hat
32867   2019-12-31   Sweden  KaggleRama  Kaggle Sticker

[6570 rows x 4 columns]


In [36]:
unseen_predictions
unseen_predictions = unseen_predictions.rename(columns={'Label': 'num_sold'})

In [37]:
#print(unseen_predictions.filter(items=['row_id', 'num_sold']))
unseen_predictions

,date,country,store,product,num_sold
row_id,,,,,
26298,2019-01-01,Finland,KaggleMart,Kaggle Mug,354.222283
26299,2019-01-01,Finland,KaggleMart,Kaggle Hat,526.498957
26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker,149.546090
26301,2019-01-01,Finland,KaggleRama,Kaggle Mug,614.682713
26302,2019-01-01,Finland,KaggleRama,Kaggle Hat,844.840085
...,...,...,...,...,...
32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat,773.191687
32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker,258.275788
32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug,940.055382


In [38]:
submit = unseen_predictions.filter(items=['row_id', 'num_sold'])

In [39]:
submit.to_csv('submit0127.csv')